In [1]:
!apt install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
make is already the newest version (4.3-4.1build1).
make set to manually installed.
xz-utils is already the newest version (5.2.5-2ubuntu1).
xz-utils set to manually installed.
curl is already the newest version (7.81.0-1ubuntu1.15).
file is already the newest version (1:5.41-3ubuntu0.1).
git is already the newest version (1:2.34.1-1ubuntu1.10).
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 24 not upgraded.
Need to get 7,367 kB of archives.
After this operation, 59.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab2 amd64 0.996-14build9 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab-dev amd64 0.996-1

In [2]:
!pip install mecab-python3==0.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mecab-python3: filename=mecab_python3-0.7-cp310-cp310-linux_x86_64.whl size=185649 sha256=53ec818140543fcbdb3fd9cb5b46f4b0169e6e0383e47d2b382f92cfe2ed6b3b
  Stored in directory: /root/.cache/pip/wheels/af/5c/2a/bb8c7ed306516444e25b06556edf0e8f0b82139b270eb4ce2d
Successfully built mecab-python3


In [3]:
import MeCab

In [4]:
text='Udemyは素晴らしい勉強だ'
m=MeCab.Tagger()
m1=m.parse(text)
m1

'Udemy\t名詞,固有名詞,組織,*,*,*,*\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n素晴らしい\t形容詞,自立,*,*,形容詞・イ段,基本形,素晴らしい,スバラシイ,スバラシイ\n勉強\t名詞,サ変接続,*,*,*,*,勉強,ベンキョウ,ベンキョー\nだ\t助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ\nEOS\n'

In [5]:
for row in m1.split("\n"):
  print(row)

Udemy	名詞,固有名詞,組織,*,*,*,*
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
素晴らしい	形容詞,自立,*,*,形容詞・イ段,基本形,素晴らしい,スバラシイ,スバラシイ
勉強	名詞,サ変接続,*,*,*,*,勉強,ベンキョウ,ベンキョー
だ	助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ
EOS



In [6]:
word_list = []
for row in m1.split("\n"):
  target_word =row.split("\t")[0]
  if target_word =="EOS":
   break
  else:
    word =row.split("\t")[1]
    if word[:2] =="名詞":
      word_list.append(target_word)


In [7]:
word_list

['Udemy', '勉強']

In [8]:
import glob
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
files =glob.glob("/content/drive/MyDrive/topic-news_sub/*.txt")

In [10]:
documents =[]
for file_path in files:
  with open(file_path,"r",encoding="utf-8") as file:
    document=file.read()
    documents.append(document)

In [11]:
documents

['http://news.livedoor.com/article/detail/5942965/\n2011-10-17T11:45:00+0900\nゆうこりん「頭おかしい」と完全否定\n10月16日に放送された日本テレビ「行列のできる法律相談所SP」で、ゆうこりんがぶっちゃけた。\n\n■TVで自分の過去を完全否定\n\n「こりん星から来た設定やめた」この発言を皮切りに、ゆうこりんは赤裸々に語り始めたのだ。\n\n“こりん星”のキャラクターを演じていたことについて、「頭おかしかった」と過去を完全否定。\n\nそれだけにとどまらない。\n\n「最初に設定作っちゃう芸能界よくない」とTVで大胆に芸能界批判まで口にする。\n\n「いちごの馬車で来たんじゃないのか」という質問にも「今日は黒い車で来た」とあっさり夢を砕くような返答。\n\n■“鬼嫁”になった小倉優子\n\n今まで知られていなかった素顔も明らかになった。\n\n親友のギャル曽根によると、実は、ゆうこりんは恐妻であるという。\n\n買い物で間違えてサイダーを買ってきた夫に「めちゃめちゃキレて怒った」というのだ。\n\n今までのゆうこりんを考えれば、放言とも言える様な発言の数々。\n\n一体何がゆうこりんを豹変させてしまったのだろうか。\n\n■前事務所との泥沼の争い\n・小倉優子と所属事務所の争いもいよいよ泥沼化！\u3000その裏で暗躍する業界人の影 - 日刊サイゾー\n・［ザイＦＸ！］小倉優子と所属事務所の間で騒動勃発！その騒動に巻き込まれたＦＸ会社とは？\n・小倉優子、事務所トラブルについて「心機一転頑張りたい」とコメント！菊地勲氏との結婚時期は「近々報告できそう」\n・小倉優子、おのろけ結婚報告！こりん星とも決別\n\n昨今では、前事務所との泥沼のトラブルが知られている。\n\n過去の自分を完全否定までする口調には、前事務所への強い不満が見えなくもない。\n\n“こりん星”を封印した後は、恐妻としてキャラを立てていくのだろうか？\n',
 'http://news.livedoor.com/article/detail/5943886/\n2011-10-17T16:16:00+0900\n反日映画か?\u3000『アジアの純真』日本公開に物議\n10月15日、映画『アジアの純真』の初日舞台あいさ

In [12]:

document_list = []
m=MeCab.Tagger()
pattern=re.compile(r'[一-龯あ-んア-ン]')
for document in documents:
  m1=m.parse(document)
  word_list = []
  for row in m1.split("\n"):
    target_word =row.split("\t")[0]
    if target_word =="EOS":
      break
    else:
      word =row.split("\t")[1]
      if word[:2] =="名詞":
        if pattern.findall(target_word)!=[]:
          word_list.append(target_word)

  words=" ".join(word_list)
  document_list.append(words)

In [13]:
document_list

['ゆうこ 頭 完全 否定 月 日 放送 日本テレビ 行列 法律 相談 所 ゆうこ 自分 過去 完全 否定 星 設定 発言 皮切り ゆうこ 赤裸々 の 星 キャラクター こと 頭 過去 完全 否定 それ 最初 設定 芸能 界 大胆 芸能 界 批判 口 いちご 馬車 ん の 質問 今日 車 夢 よう 返答 鬼 嫁 小倉 優子 今 素顔 明らか 親友 ギャル 曽根 ゆうこ 恐妻 買い物 サイダー 夫 めちゃめちゃ キレ の 今 ゆうこ 放言 様 発言 数々 何 ゆうこ 豹変 の 事務所 泥沼 争い 小倉 優子 所属 事務所 争い 泥沼 化 裏 暗躍 業界 人 影 日刊 サイゾー ザイ 小倉 優子 所属 事務所 間 騒動 勃発 騒動 会社 小倉 優子 事務所 トラブル 心機一転 コメント 菊地 勲 氏 結婚 時期 報告 そう 小倉 優子 結婚 報告 星 決別 昨今 事務所 泥沼 トラブル 過去 自分 完全 否定 口調 事務所 不満 星 封印 後 恐妻 キャラ の',
 '反日 映画 アジア 純真 日本 公開 物議 月 日 映画 アジア 純真 初日 舞台 あいさつ 都内 主演 韓 英恵 笠井 しげ 嶋 一貴 監督 ら 登場 あいさつ 内容 ネット 上 物議 関連 記事 多く 映画 館 上映 拒否 反日 映画 青春 映画 ロード ムービー 歳 韓 英恵 舞台 あいさつ 本 作 年 北朝鮮 拉致 事件 朝鮮 感情 蔓延 頃 日本 舞台 日本人 チンピラ 姉 在日 朝鮮 人 少女 現場 フリ 日本人 男子 高校生 差別 テロ 社会 復讐 ストーリー 映画 過激 内容 年 製作 多く 映画 館 上映 拒否 日本 公開 主演 韓 映画 ネット よう 反日 映画 わたし 自身 青春 映画 ロード ムービー わたし 代 最後 思い出 映画 アジア 純真 笑顔 これ ネット ユーザー パッチギ 反日 八つ 当たり 純真 チンピラ 復讐 日本 国籍 竹島 日本 固有 領土 宣言 あちら 決別 こと 冷ややか 反応 映画 タイトル アジア 純真 日本 曲名 こと タイトル 許可 声 主演 韓 才 時 鈴木 清 順 監督 作 ピストル オペラ デビュー 以後 誰 疾走 悪人 ほか 数々 作品 出演 着実 キャリア 個性 派 女優 アジア 純真 評判 散々 もの 回 ロッテルダム 国際 映画 祭 

In [14]:
tfidf=TfidfVectorizer()
tfidf_metrix=tfidf.fit_transform(document_list)

In [15]:
pd.DataFrame(tfidf_metrix.toarray())

,0,1,2,3,4,5,6,7,8,9,...,3284,3285,3286,3287,3288,3289,3290,3291,3292,3293
0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.168763,0.0,0.056254,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
def cos_sim(vec_a,vec_b):
  dot=np.dot(vec_a,vec_b)
  norm_a=np.linalg.norm(vec_a)
  norm_b=np.linalg.norm(vec_b)
  return dot/(norm_a*norm_b)

vec_a=np.array([1,2,3])
vec_b=np.array([4,5,6])

cos_sim(vec_a,vec_b)

0.9746318461970762

In [17]:
num = tfidf_metrix.shape[0]
cos_sim_matrix=np.zeros((num,num))

for i in range(num):
  for j in range(num):
    vec_a = tfidf_metrix[i,:].toarray().flatten()
    vec_b = tfidf_metrix[i,:].toarray().flatten()

    cos_sim_matrix[i,j] = cos_sim(vec_a,vec_b)

In [18]:
tfidf_metrix[1,:].toarray().flatten().shape

(3294,)

In [19]:
pd.DataFrame(cos_sim_matrix)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
96,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
97,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
98,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [20]:
cos_sim_matrix=cosine_similarity(tfidf_metrix)

In [21]:
pd.DataFrame(cos_sim_matrix)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.007153,0.011856,0.006532,0.004458,0.012895,0.009663,0.001349,0.045027,0.015673,...,0.002446,0.000000,0.002315,0.018626,0.081629,0.008901,0.039696,0.008899,0.008605,0.039960
1,0.007153,1.000000,0.017972,0.030710,0.043606,0.012238,0.057134,0.009761,0.006019,0.017739,...,0.009851,0.011828,0.041074,0.009509,0.005223,0.008396,0.012451,0.005365,0.021888,0.008052
2,0.011856,0.017972,1.000000,0.023557,0.037365,0.008355,0.025489,0.004858,0.005555,0.011531,...,0.029544,0.008243,0.052265,0.002658,0.003098,0.070434,0.013335,0.189264,0.024834,0.044311
3,0.006532,0.030710,0.023557,1.000000,0.007589,0.035036,0.005356,0.012242,0.027032,0.040502,...,0.011081,0.000000,0.007850,0.024435,0.033669,0.009209,0.050209,0.010295,0.019583,0.021366
4,0.004458,0.043606,0.037365,0.007589,1.000000,0.010955,0.088307,0.003623,0.004283,0.074650,...,0.017864,0.000000,0.017257,0.000990,0.002095,0.023546,0.000846,0.012787,0.014659,0.024462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.008901,0.008396,0.070434,0.009209,0.023546,0.039004,0.012667,0.005871,0.007052,0.012205,...,0.031460,0.008375,0.023469,0.001528,0.011326,1.000000,0.007141,0.010961,0.003808,0.027402
96,0.039696,0.012451,0.013335,0.050209,0.000846,0.007224,0.004357,0.001914,0.162508,0.013532,...,0.000636,0.000000,0.005108,0.010104,0.149487,0.007141,1.000000,0.004071,0.007450,0.020745
97,0.008899,0.005365,0.189264,0.010295,0.012787,0.007044,0.004468,0.010540,0.003891,0.001868,...,0.007283,0.005562,0.008908,0.003804,0.006899,0.010961,0.004071,1.000000,0.014734,0.004520
98,0.008605,0.021888,0.024834,0.019583,0.014659,0.008789,0.015340,0.002346,0.007380,0.016922,...,0.007145,0.027091,0.025327,0.000000,0.003574,0.003808,0.007450,0.014734,1.000000,0.003791


In [22]:
array=np.array([
[1,2,3],
 [4,5,6],
[7,8,9]
])

np.unravel_index(np.argmax(array),array.shape)

(2, 2)

In [23]:
array

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [24]:
np.fill_diagonal(cos_sim_matrix,0)

In [25]:
doc_index =np.unravel_index(np.argmax(cos_sim_matrix),cos_sim_matrix.shape)

In [26]:
cos_sim_matrix[doc_index]

0.6108464979109195

In [27]:
documents[66]

'http://news.livedoor.com/article/detail/5933360/\n2011-10-13T11:29:00+0900\nメディアに情報操作されるウォール街占拠デモの実態\n\u3000米ニューヨークのウォール街で始まった抗議デモは市北部への拡大にとどまらず、大手金融機関経営者や著名資産家の自宅にまでも及びつつある。法外な報酬を受け取る金融マンたちを非難し、広がりつつある格差の溝を強調する今回のデモ活動。各紙はこういった動きを連日取り上げ、さながら勇敢な主人公が諸悪の根源に立ち向かう正義の物語のように書き立てている。\n\n\u3000しかし、ウォール街のデモがここまで長期化し拡大を続けている理由は、本当に「彼らが格差の是正を求めているから」なのだろうか。一部では、メディアの報道とはあまりにもかけ離れたデモの実態を指摘する動きも見られる。\n\n\u3000人目も憚らず行われる若者たちのフリーセックスや食事目当てのホームレス。不衛生極まりない状況下に集まった人々は、とても反社会的な意思表明をしにきている様には見えない。チュニジアで端を発した"ジャスミン革命"とは似て非なるものだ。\n\n（関連リンク）\n「ウォール街占拠デモ」はどこまで本気なのか？\nウォール街で働く人々、私たちの生活はそんなに甘いものではない—米華字紙\nMail Online（英語）\n'

In [28]:
documents[77]

'http://news.livedoor.com/article/detail/5913367/\n2011-10-05T11:53:00+0900\n韓国がハブられたiPhone4S発表会\n\u3000米アップルが4日に発表したiPhone4S。iPhone5への期待を裏切り、一時株価にまで影響を与えたこの発表会において、ひそかにもうひとつ話題になっていることがある。\n\n\u3000それはiPhone4Sの各国での発売日を発表した場面。ジョブズ氏の後継者、クック氏の背後にあるスクリーンに映し出された数々の国名の中に、"Korea"の文字がどこにもなかったのだ。\n\nこれにより「アップルが韓国でのiPhone4Sの供給を拒否?」「韓国はハブられた」と一部で話題になっている。\n\n\u3000サムスンとアップルの間にはこれまで、アップルがiPhone3Gにデザインが酷似しているとしてGalaxySを一部の国で販売できないようにしてしまったり、サムスンが"iPhone5"の発売と同時に韓国国内での販売差し止めを求める方針であることを明らかにするなど、激しい争いがあった。\n\n・「Galaxyはパクリ」とアップル提訴。サムスンは「法的手段で対抗する」\n・特許戦争はアップル優勢。欧州でGalaxy S、S IIに販売停止仮命令\n・もしや次はアップルが有罪に？\u3000サムスンのGalaxy Tabを販売停止に追い込んだ訴訟で、アップルがでっち上げ証拠を提出した疑惑...\n・アップルとサムスンは犬猿の仲ですが、iPhone 4の部品の多くはサムスン製だということを忘れないで\n・泥沼の訴訟合戦から見えてくる アップルとサムスンの関係変化 \n\n\u3000今回のiPhone4S発表会で、販売国名の中に韓国の名前がなかったのはこの両者の関係悪化が原因だと予想される。\n\n\u3000そしてこのiPhone4Sの発表と同時に、サムスンは同機種の販売禁止を求める仮処分申請を、海外の裁判所で行うことを検討していると報道された。\n\n・新型iPhoneの販売禁止申し立て、サムスンが反撃へ \n\n\u3000更なるドロ沼に深くはまりこんでいくアップル対サムスンの争い。iPhone、Galaxyという人気を二分する機種が手に入らなくなることで一番がっかりするの

In [29]:
import glob
import MeCab
from gensim.models import Word2Vec

In [30]:
stop_words = [
    "あそこ", "あたり", "あちら", "あっち", "あと", "あな", "あなた", "あれ", "いくつ",
    "いつ", "いま", "いや", "いろいろ", "うち", "おおまか", "おまえ", "おれ", "がい",
    "かく", "かたち", "かやの", "から", "がら", "きた", "くせ", "ここ", "こっち", "こと",
    "ごと", "こちら", "ごっちゃ", "これ", "これら", "ごろ", "さまざま", "さらい", "さん",
    "しかた", "しよう", "すか", "ずつ", "すね", "すべて", "ぜんぶ", "そう", "そこ",
    "そちら", "そっち", "そで", "それ", "それぞれ", "それなり", "たくさん", "たち",
    "たび", "ため", "だめ", "ちゃ", "ちゃん", "てん", "とおり", "とき", "どこ", "どこか",
    "ところ", "どちら", "どっか", "どっち", "どれ", "なか", "なかば", "なに", "など",
    "なん", "はじめ", "はず", "はるか", "ひと", "ひとつ", "ふく", "ぶり", "べつ", "へん",
    "ぺん", "ほう", "ほか", "まさ", "まし", "まとも", "まま", "みたい", "みつ",
    "みなさん", "みんな", "もと", "もの", "もん", "やつ", "よう", "よそ", "わけ",
    "わたし", "ハイ", "上", "中", "下", "字", "年", "月", "日", "時", "分", "秒", "週",
    "火", "水", "木", "金", "土", "国", "都", "道", "府", "県", "市", "区", "町", "村",
    "各", "第", "方", "何", "的", "度", "文", "者", "性", "体", "人", "他", "今", "部",
    "課", "係", "外", "類", "達", "気", "室", "口", "誰", "用", "界", "会", "首", "男",
    "女", "別", "話", "私", "屋", "店", "家", "場", "等", "見", "際", "観", "段", "略",
    "例", "系", "論", "形", "間", "地", "員", "線", "点", "書", "品", "力", "法", "感",
    "作", "元", "手", "数", "彼", "彼女", "子", "内", "楽", "喜", "怒", "哀", "輪", "頃",
    "化", "境", "俺", "奴", "高", "校", "婦", "伸", "紀", "誌", "レ", "行", "列", "事",
    "士", "台", "集", "様", "所", "歴", "器", "名", "情", "連", "毎", "式", "簿", "回",
    "匹", "個", "席", "束", "歳", "目", "通", "面", "円", "玉", "枚", "前", "後", "左",
    "右", "次", "先", "春", "夏", "秋", "冬", "一", "二", "三", "四", "五", "六", "七",
    "八", "九", "十", "百", "千", "万", "億", "兆", "下記", "上記", "時間", "今回",
    "前回", "場合", "一つ", "年生", "自分", "ヶ所", "ヵ所", "カ所", "箇所", "ヶ月",
    "ヵ月", "カ月", "箇月", "名前", "本当", "確か", "時点", "全部", "関係", "近く",
    "方法", "我々", "違い", "多く", "扱い", "新た", "その後", "半ば", "結局", "様々",
    "以前", "以後", "以降", "未満", "以上", "以下", "幾つ", "毎日", "自体", "向こう",
    "何人", "手段", "同じ", "感じ"
]

In [31]:
files=glob.glob("/content/drive/MyDrive/topic-news_sub/*.txt")
documents=[]
for file_path in files:
  with open(file_path,"r",encoding="utf-8")as file:
    document=file.read()
    documents.append(document)

document_list = []
m = MeCab.Tagger()
pattern = re.compile(r'[一-龯あ-んア-ン]')

for document in documents:
  m1=m.parse(document)
  word_list = []
  for row in m1.split("\n"):
    target_word = row.split("\t")[0]
    if target_word == "EOS":
      break
    else:
      word = row.split("\t")[1]
      if word[:2] == "名詞":
        if pattern.findall(target_word) != [] and target_word not in stop_words:
          word_list.append(target_word)

  document_list.append(word_list)


In [32]:
document_list

[['ゆうこ',
  '頭',
  '完全',
  '否定',
  '放送',
  '日本テレビ',
  '行列',
  '法律',
  '相談',
  'ゆうこ',
  '過去',
  '完全',
  '否定',
  '星',
  '設定',
  '発言',
  '皮切り',
  'ゆうこ',
  '赤裸々',
  'の',
  '星',
  'キャラクター',
  '頭',
  '過去',
  '完全',
  '否定',
  '最初',
  '設定',
  '芸能',
  '大胆',
  '芸能',
  '批判',
  'いちご',
  '馬車',
  'ん',
  'の',
  '質問',
  '今日',
  '車',
  '夢',
  '返答',
  '鬼',
  '嫁',
  '小倉',
  '優子',
  '素顔',
  '明らか',
  '親友',
  'ギャル',
  '曽根',
  'ゆうこ',
  '恐妻',
  '買い物',
  'サイダー',
  '夫',
  'めちゃめちゃ',
  'キレ',
  'の',
  'ゆうこ',
  '放言',
  '発言',
  '数々',
  'ゆうこ',
  '豹変',
  'の',
  '事務所',
  '泥沼',
  '争い',
  '小倉',
  '優子',
  '所属',
  '事務所',
  '争い',
  '泥沼',
  '裏',
  '暗躍',
  '業界',
  '影',
  '日刊',
  'サイゾー',
  'ザイ',
  '小倉',
  '優子',
  '所属',
  '事務所',
  '騒動',
  '勃発',
  '騒動',
  '会社',
  '小倉',
  '優子',
  '事務所',
  'トラブル',
  '心機一転',
  'コメント',
  '菊地',
  '勲',
  '氏',
  '結婚',
  '時期',
  '報告',
  '小倉',
  '優子',
  '結婚',
  '報告',
  '星',
  '決別',
  '昨今',
  '事務所',
  '泥沼',
  'トラブル',
  '過去',
  '完全',
  '否定',
  '口調',
  '事務所',
  '不満',
  '星',
  '封印',
  '恐妻',
  'キャラ',
  'の'],
 [

In [33]:
model=Word2Vec(document_list,min_count=1)

In [34]:
cos_sim(model.wv["放送"],model.wv["視聴"])

0.1318814

In [35]:
model.wv.most_similar(["視聴"],topn=5)

[('記事', 0.411211222410202),
 ('韓', 0.38789480924606323),
 ('風評', 0.37898731231689453),
 ('位', 0.3731723427772522),
 ('意見', 0.3716464340686798)]

In [36]:
sentence=["私", "は", "報道", "アナウンサー", "です"]

word_vec=[]
for word in sentence:
  if word in model.wv:
    word_vec.append(model.wv[word])

In [37]:
np.array(word_vec).shape
np.mean(word_vec,axis=0)

array([ 3.67738702e-03,  3.95723106e-03,  3.58017581e-03,  1.55368820e-03,
        5.19832503e-03, -5.81057463e-03, -4.08322643e-03,  4.02586535e-04,
        3.19457357e-03, -6.16418663e-03,  4.19688877e-04, -3.97665240e-03,
        4.18569939e-03,  2.63237534e-03, -5.43882605e-04, -2.89308256e-03,
       -9.90973786e-05, -4.38929256e-03, -4.68868017e-03, -2.82914774e-03,
       -1.94280676e-03,  8.11370648e-03,  2.32651411e-03,  6.10502157e-03,
        2.10614083e-03, -8.58854153e-04, -1.25230732e-03,  1.72179122e-03,
       -7.84465205e-03, -1.29825203e-04,  2.68111192e-03, -5.84213436e-03,
        4.04127699e-04,  3.53342923e-03, -2.43522529e-03,  8.86307005e-03,
       -4.82894946e-04,  1.64201204e-03,  3.73008661e-03, -2.56166328e-04,
        2.41819676e-03, -6.00373279e-03, -3.64297838e-03,  3.90665466e-03,
       -5.64425252e-04, -1.11629982e-02, -6.32054918e-03, -8.42788722e-05,
        6.15113042e-03,  1.19642448e-03,  2.39694724e-04, -3.04503832e-03,
       -4.93281404e-04, -

In [38]:
def cal_vec(sentence,model):
  word_vec=[model.wv[word] for word in sentence if word in model.wv]
  sentence_vec=np.mean(word_vec,axis=0)
  return sentence_vec

vec1=cal_vec(document_list[66],model)
vec2=cal_vec(document_list[77],model)

In [39]:
cos_sim(vec1,vec2)

0.7224799

In [40]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument

In [41]:
tagged_data=[]
for i, doc in enumerate(document_list):
  tagged_data.append(TaggedDocument(words=doc,tags=[str(i)]))


In [42]:
tagged_data

[TaggedDocument(words=['ゆうこ', '頭', '完全', '否定', '放送', '日本テレビ', '行列', '法律', '相談', 'ゆうこ', '過去', '完全', '否定', '星', '設定', '発言', '皮切り', 'ゆうこ', '赤裸々', 'の', '星', 'キャラクター', '頭', '過去', '完全', '否定', '最初', '設定', '芸能', '大胆', '芸能', '批判', 'いちご', '馬車', 'ん', 'の', '質問', '今日', '車', '夢', '返答', '鬼', '嫁', '小倉', '優子', '素顔', '明らか', '親友', 'ギャル', '曽根', 'ゆうこ', '恐妻', '買い物', 'サイダー', '夫', 'めちゃめちゃ', 'キレ', 'の', 'ゆうこ', '放言', '発言', '数々', 'ゆうこ', '豹変', 'の', '事務所', '泥沼', '争い', '小倉', '優子', '所属', '事務所', '争い', '泥沼', '裏', '暗躍', '業界', '影', '日刊', 'サイゾー', 'ザイ', '小倉', '優子', '所属', '事務所', '騒動', '勃発', '騒動', '会社', '小倉', '優子', '事務所', 'トラブル', '心機一転', 'コメント', '菊地', '勲', '氏', '結婚', '時期', '報告', '小倉', '優子', '結婚', '報告', '星', '決別', '昨今', '事務所', '泥沼', 'トラブル', '過去', '完全', '否定', '口調', '事務所', '不満', '星', '封印', '恐妻', 'キャラ', 'の'], tags=['0']),
 TaggedDocument(words=['反日', '映画', 'アジア', '純真', '日本', '公開', '物議', '映画', 'アジア', '純真', '初日', '舞台', 'あいさつ', '都内', '主演', '韓', '英恵', '笠井', 'しげ', '嶋', '一貴', '監督', 'ら', '登場', 'あいさつ', '内容', 'ネット', '物議', '関連', '記事', '映画

In [43]:
model=Doc2Vec(documents=tagged_data,min_count=1,epochs=100)

In [44]:
model.dv.most_similar([model.dv[66]],topn=5)

[('66', 1.0000001192092896),
 ('64', 0.6540166139602661),
 ('26', 0.6363738179206848),
 ('84', 0.5635775327682495),
 ('43', 0.5610743761062622)]

In [45]:
documents[24]

'http://news.livedoor.com/article/detail/5934833/\n2011-10-13T18:12:00+0900\n「韓国人の顔は何故醜い？」に韓国人激怒\n2ちゃんねるに作られたスレッド「韓国人は何故ブサイクなのか？」に寄せられた意見や批判をソウル新聞が取り上げられたことで、韓国ネットユーザーから日本のネットユーザーへの批判が相次いでいる。\n\n取り上げられた内容は、「顔が大きい」「下顎が広い」「鼻が丸い」「眉間が広い」「下唇が上唇よりも出ている」「目が小さくて二重まぶたがない」「頬骨がついている」など、韓国人の顔の特徴を悪意をもって貶めたものがほとんど。さらに、「韓国人たちの劣悪な遺伝子に問題がある」「韓国人は性格が悪い」など、容姿に関係のない悪意あるコメントが次々と並んでいた。\n\n同紙では、これはごく一部の日本人が持っている嫌悪感を示す事例と説明。日本国内で韓流ブームが起こり、日本人の韓国に対するイメージが徐々に良くなることを恐れ、これを阻止しようとする人々の声が一段と高まっていると続けた。\n\nまた、韓流の主役たちに対する悪質な攻撃にも強さを高めている事にも触れ、日本で人気の高いガールズグループ、『少女時代』や『KARA』が"奴隷契約と性上納を使って成功した"など、悪質な噂話に苦しめられたのが代表例とも述べている。\n\nこの内容に激怒した韓国内ネチズンたちも、日本を卑下してばかにするコメントで対抗しており、インターネット上では両国民の対立が深刻化する懸念もある状態だ。中には"世界的に日本人の顔が最悪ではないか？自分たちの顔は、考えもせずにとんでもない写真だけを選んでいる"といったコメントも。\n\n一部の日本人による悪意あるコメントが紹介され、日韓の関係が悪化することはなるべく避けたいものだ。\n\n・“한국인들이 못생긴 이유는…” 도를 넘은 日 ‘혐한 열풍’ - ソウル新聞\n'

In [46]:
documents[4]

'http://news.livedoor.com/article/detail/5955767/\n2011-10-21T11:48:00+0900\nチャン・グンソクの“乱入”に批判が殺到\nチャン・グンソクの、授賞式における振舞いが韓国で非難されている。\n\n17日に行われた韓国の大鐘賞映画祭で、受賞に感極まり涙を流している女優のキム・ハヌルのそばで、無関係なチャン・グンソクが乱入しVサインやウィンクをしたのだ。キム・ハヌルはグンソクとは次作の共演者という関係だが、グンソクとの共演作で受賞したわけではないので、韓国のネットユーザーからは「非常識」で「配慮が無い」と批判が殺到した。\n\nこの批判に対して、チャン・グンソクはTwitterで反応。「なぜウィンクをしたんだろう」と反省の呟きと同時に投稿した画像は何故か目のアップ。\n\n思わずぎょっとするような画像を載せる“奇行”に、「非常識な振る舞いを反省していない」とさらなる批判を呼んでいる。\n\n【関連情報】\n・チャン・グンソクの釈明のツイート\nhttps://twitter.com/#!/jksjapan/status/126089605031272448\nhttps://twitter.com/#!/jksjapan/status/126926405173907456\nhttps://twitter.com/#!/jksjapan/status/126925157515280384\n・장근석, 우는 김하늘 옆에서 윙크 논란에 "왜 했을까?" 해명（마이데일리）\nhttp://www.mydaily.co.kr/news/read.html?newsid=201110182030491110\n【関連記事】\n・大人気なチャン・グンソクのファンがまわりに一人もいない理由\nhttp://news.livedoor.com/article/detail/5949379/\n'

In [53]:
def tokenize(document):
  m1=m.parse(document)
  word_list = []
  for row in m1.split("\n"):
    target_word =row.split("\t")[0]
    if target_word =="EOS":
      break
    else:
      word =row.split("\t")[1]
      if word[:2] =="名詞":
        if pattern.findall(target_word)!=[]:
          word_list.append(target_word)
  return word_list

new_doc="新しい文章がはいります"
tokenize_noc=tokenize(new_doc)
vector=model.infer_vector(tokenize_noc)
vector

array([-0.00638896, -0.07650442,  0.10354596, -0.05244625,  0.02016247,
       -0.12349533,  0.05028916,  0.12007722, -0.33233854, -0.11149474,
       -0.16951022, -0.22737187, -0.01514966,  0.10839418, -0.06334655,
       -0.1189514 ,  0.08976197, -0.15221693,  0.04965505, -0.1731718 ,
        0.13536128,  0.15065676,  0.15730524,  0.1374209 ,  0.00167466,
       -0.17560148, -0.337165  , -0.15981649, -0.10326165, -0.09041556,
        0.29897332,  0.36478975, -0.00228109,  0.11625817,  0.01602896,
        0.12628825,  0.16227998, -0.35645148, -0.07122891, -0.03470719,
        0.00778516, -0.13904957, -0.08641249, -0.21719305,  0.1041541 ,
       -0.01124579, -0.05619791, -0.03221799,  0.02603935,  0.16822648,
       -0.18995067, -0.09076573,  0.05945903,  0.02755881, -0.19377114,
        0.07438217, -0.03006465, -0.05813966, -0.14438805, -0.11782686,
        0.12525669,  0.10011077, -0.24904472, -0.03308554,  0.04458783,
        0.18799789, -0.06927823,  0.085959  , -0.11885548,  0.13